In [1]:
import datetime as dt

import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_absolute_error
from sklearn.preprocessing import (
    OneHotEncoder,
    MinMaxScaler,
    StandardScaler
)
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from analytics.machine_learning.price_prediction_with_fundamentals import utils

# Random Forrest

In [2]:
from sklearn.ensemble import RandomForestRegressor

dataset = utils.get_dataset()

train_set, test_set = utils.split_data_to_train_and_test(
    df=dataset,
    cutoff_date=dt.datetime(2023,6,1)
)

cols_to_drop = ['symbol', 'fiscal_date_ending', 'avg_next_three_months_price']

y_train = train_set[['avg_next_three_months_price', 'sector']]
X_train = train_set.drop(cols_to_drop, axis=1)

y_test = test_set[['avg_next_three_months_price', 'sector']]
X_test = test_set.drop(cols_to_drop, axis=1)

column_transformer = make_column_transformer(
    (
        OneHotEncoder(), ['sector']
    ),
    (
        StandardScaler(),
        ~X_train.columns.isin(['sector'])
    ),
    remainder='passthrough'
)

random_forrest_regressor = make_pipeline(
    column_transformer,
    RandomForestRegressor(n_estimators=100)
)

random_forrest_regressor.fit(X_train, y_train['avg_next_three_months_price'])
y_pred_random_forrest = random_forrest_regressor.predict(X_test)

print("Mean absolute error: %.2f" % mean_absolute_error(y_test['avg_next_three_months_price'], y_pred_random_forrest))
print("Coefficient of determination: %.2f" % r2_score(y_test['avg_next_three_months_price'], y_pred_random_forrest))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['avg_next_three_months_price'], y_pred_random_forrest))


Mean absolute error: 6.26
Coefficient of determination: 0.97
Mean absolute pct error: 0.13


# Lasso Regression

In [2]:
from sklearn.linear_model import Lasso

dataset = utils.get_dataset()

train_set, test_set = utils.split_data_to_train_and_test(
    df=dataset,
    cutoff_date=dt.datetime(2023,6,1)
)

cols_to_drop = ['symbol', 'fiscal_date_ending', 'avg_next_three_months_price']


y_train = train_set[['avg_next_three_months_price', 'sector']]
X_train = train_set.drop(cols_to_drop, axis=1)

y_test = test_set[['avg_next_three_months_price', 'sector']]
X_test = test_set.drop(cols_to_drop, axis=1)


column_transformer = make_column_transformer(
    (
        OneHotEncoder(), ['sector']
    ),
    (
        StandardScaler(),
        ~X_train.columns.isin(['sector'])
    ),
    remainder='passthrough'
)

lasso_reg = make_pipeline(
    column_transformer,
    Lasso(max_iter=3000, alpha=1)
)

lasso_reg.fit(X_train, y_train['avg_next_three_months_price'])
y_pred_lasso_reg =  lasso_reg.predict(X_test)

print("Mean absolute error: %.2f" % mean_absolute_error(y_test['avg_next_three_months_price'], y_pred_lasso_reg))
print("Coefficient of determination: %.2f" % r2_score(y_test['avg_next_three_months_price'], y_pred_lasso_reg))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['avg_next_three_months_price'], y_pred_lasso_reg))

Mean absolute error: 6.01
Coefficient of determination: 0.99
Mean absolute pct error: 0.12


# XGBoost

In [3]:
import xgboost as xgb

dataset = utils.get_dataset()

train_set, test_set = utils.split_data_to_train_and_test(
    df=dataset,
    cutoff_date=dt.datetime(2023,6,1)
)

cols_to_drop = ['symbol', 'fiscal_date_ending', 'avg_next_three_months_price']


y_train = train_set[['avg_next_three_months_price', 'sector']]
X_train = train_set.drop(cols_to_drop, axis=1)

y_test = test_set[['avg_next_three_months_price', 'sector']]
X_test = test_set.drop(cols_to_drop, axis=1)


column_transformer = make_column_transformer(
    (
        OneHotEncoder(), ['sector']
    ),
    (
        StandardScaler(),
        ~X_train.columns.isin(['sector'])
    ),
    remainder='passthrough'
)

xgb_reg = make_pipeline(
    column_transformer,
    xgb.XGBRegressor(
        objective = 'reg:squarederror',
        n_estimators = 100,
        learning_rate = 0.1,
    )
)

xgb_reg.fit(X_train, y_train['avg_next_three_months_price'])
y_pred_xgb_reg =  xgb_reg.predict(X_test)

print("Mean absolute error: %.2f" % mean_absolute_error(y_test['avg_next_three_months_price'], y_pred_xgb_reg))
print("Coefficient of determination: %.2f" % r2_score(y_test['avg_next_three_months_price'], y_pred_xgb_reg))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['avg_next_three_months_price'], y_pred_xgb_reg))


Mean absolute error: 7.30
Coefficient of determination: 0.94
Mean absolute pct error: 0.14


# GPBoost

In [2]:
import gpboost as gpb

dataset = utils.get_dataset()

train_set, test_set = utils.split_data_to_train_and_test(
    df=dataset,
    cutoff_date=dt.datetime(2023,6,1)
)

train_group = train_set['symbol']
test_group = test_set['symbol']

cols_to_drop = ['symbol', 'fiscal_date_ending', 'avg_next_three_months_price']

y_train = train_set[['avg_next_three_months_price', 'sector']]
X_train = train_set.drop(cols_to_drop, axis=1)

y_test = test_set[['avg_next_three_months_price', 'sector']]
X_test = test_set.drop(cols_to_drop, axis=1)


column_transformer = make_column_transformer(
    (
        OneHotEncoder(), ['sector']
    ),
    (
        StandardScaler(),
        ~X_train.columns.isin(['sector'])
    ),
    remainder='passthrough'
)

# Fit and transform the training data
X_train_transformed = column_transformer.fit_transform(X_train)
# Transform the test data
X_test_transformed = column_transformer.transform(X_test)

gp_model = gpb.GPModel(group_data=train_group, likelihood="gaussian")
params = {'learning_rate': 0.01, 'max_depth': 3, 'num_leaves': 2**10, 'verbose': 0}

data_train = gpb.Dataset(X_train_transformed, y_train['avg_next_three_months_price'])
gpb_reg = gpb.train(params=params, train_set=data_train, gp_model=gp_model, num_boost_round=100)

y_pred_gpb_reg =  gpb_reg.predict(X_test_transformed, group_data_pred=test_group)['response_mean']

print("Mean absolute error: %.2f" % mean_absolute_error(y_test['avg_next_three_months_price'], y_pred_gpb_reg))
print("Coefficient of determination: %.2f" % r2_score(y_test['avg_next_three_months_price'], y_pred_gpb_reg))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['avg_next_three_months_price'], y_pred_gpb_reg))


Mean absolute error: 21.21
Coefficient of determination: 0.76
Mean absolute pct error: 0.61


# SVR

In [2]:
from sklearn.svm import SVR

dataset = utils.get_dataset()

train_set, test_set = utils.split_data_to_train_and_test(
    df=dataset,
    cutoff_date=dt.datetime(2023,6,1)
)

cols_to_drop = ['symbol', 'fiscal_date_ending', 'avg_next_three_months_price']


y_train = train_set[['avg_next_three_months_price', 'sector']]
X_train = train_set.drop(cols_to_drop, axis=1)

y_test = test_set[['avg_next_three_months_price', 'sector']]
X_test = test_set.drop(cols_to_drop, axis=1)


column_transformer = make_column_transformer(
    (
        OneHotEncoder(), ['sector']
    ),
    (
        StandardScaler(),
        ~X_train.columns.isin(['sector'])
    ),
    remainder='passthrough'
)

svr_reg = make_pipeline(
    column_transformer,
    SVR()
)

svr_reg.fit(X_train, y_train['avg_next_three_months_price'])
y_pred_svr_reg =  svr_reg.predict(X_test)

print("Mean absolute error: %.2f" % mean_absolute_error(y_test['avg_next_three_months_price'], y_pred_svr_reg))
print("Coefficient of determination: %.2f" % r2_score(y_test['avg_next_three_months_price'], y_pred_svr_reg))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['avg_next_three_months_price'], y_pred_svr_reg))


Mean absolute error: 16.46
Coefficient of determination: 0.36
Mean absolute pct error: 0.36
